<a href="https://colab.research.google.com/github/Matonice/Advance_NLP/blob/main/Build_Model_On_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Read in data, clean it, and then split it into train and test split
import gensim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option("display.max_colwidth", 100)

messages = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None)
#messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
messages.columns = ["label", "text"]
messages["text_clean"] = messages["text"].apply(lambda x: gensim.utils.simple_preprocess(x))

x_train, x_test, y_train, y_test = train_test_split(messages["text_clean"], messages["label"], test_size=0.2)

In [6]:
#Train the word2vec model
w2v_model = gensim.models.Word2Vec(x_train,
                                   size=100,
                                   window=5,
                                   min_count=2)

In [10]:
# Replace the words in each text message with the learned word vector
words = set(w2v_model.wv.index2word)
x_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in x_train])

x_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in x_test])

<ipython-input-10-313dc9e94642>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
<ipython-input-10-313dc9e94642>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


In [11]:
# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not  learn any of the words in the text message during training)

x_train_vect_avg = []
for v in x_train_vect:
  if v.size:
    x_train_vect_avg.append(v.mean(axis=0))
  else:
    x_train_vect_avg.append(np.zeros(100, dtype=float))

x_test_vect_avg = []
for v in x_test_vect:
  if v.size:
    x_test_vect_avg.append(v.mean(axis=0))
  else:
    x_test_vect_avg.append(np.zeros(100, dtype=float))

In [13]:
# What does the unaveraged version  look like?
x_train_vect[0]

array([[-7.61496961e-01, -1.12423502e-01, -6.46253109e-01,
        -3.22488725e-01,  7.52294898e-01, -1.43379956e-01,
        -2.43332162e-01, -1.45564213e-01, -5.01384854e-01,
        -3.39190334e-01, -1.34143591e-01, -4.52799946e-01,
        -6.06232509e-03, -4.38508540e-01,  6.24794483e-01,
        -1.78497016e-01, -1.66599587e-01,  1.42685041e-01,
         1.38553545e-01,  2.86336720e-01, -1.19575046e-01,
        -2.02508926e-01, -9.06872928e-01, -5.67761421e-01,
         2.60765702e-01,  1.41192928e-01, -1.95978885e-03,
        -8.58795494e-02,  1.02583431e-01,  2.15017393e-01,
         4.69031215e-01,  3.35613728e-01, -3.71601013e-03,
        -7.68442810e-01, -8.71047825e-02,  1.77912965e-01,
         7.73388326e-01, -2.95008689e-01,  7.59604096e-01,
        -1.00808538e-01,  8.49306524e-01,  9.24426079e-01,
         6.26253545e-01, -4.94991653e-02,  1.05475044e+00,
         1.77231580e-01, -8.33229497e-02,  7.31807351e-02,
         4.31409568e-01, -5.53755425e-02, -2.63689935e-0

In [14]:
# What does the averaged version look like?
x_train_vect_avg[0]

array([-0.6737533 , -0.10462996, -0.5890326 , -0.2841587 ,  0.66573066,
       -0.13410401, -0.22447047, -0.1376764 , -0.46016514, -0.3036722 ,
       -0.12226097, -0.4055258 , -0.00900481, -0.40104946,  0.5487741 ,
       -0.16779104, -0.15907308,  0.13301449,  0.13296458,  0.25946307,
       -0.09919976, -0.17556375, -0.82889795, -0.5033593 ,  0.220113  ,
        0.12286819, -0.00551319, -0.08549438,  0.08985383,  0.17962886,
        0.4288296 ,  0.29571933,  0.0039035 , -0.7079184 , -0.08835936,
        0.15551893,  0.69989645, -0.27601779,  0.6743805 , -0.08846341,
        0.7713042 ,  0.82743114,  0.57292366, -0.02745052,  0.9564999 ,
        0.16984357, -0.07733747,  0.0684287 ,  0.40751067, -0.05555758,
       -0.24172291,  0.48289004,  0.46306378, -0.1914636 ,  0.33612517,
       -0.6929883 , -0.2836606 , -0.16958019, -0.31362286, -0.3441004 ,
        1.3513747 ,  0.6243583 ,  0.06889493, -0.11770655,  0.1777036 ,
        0.02802414, -0.61264944,  0.17628396,  0.56399673, -0.28

## Fit RandomForestClassifier On Top Of Word Vectors

In [15]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(x_train_vect_avg, y_train.values.ravel())

In [16]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(x_test_vect_avg)

In [19]:
# Evaluate the predictions of the model on the holdout test set
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred, pos_label="spam")
recall = recall_score(y_test, y_pred, pos_label="spam")
print("precision: {} / Recall: {} / Accuracy: {}".format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)
))

precision: 0.918 / Recall: 0.747 / Accuracy: 0.957
